In [1]:
import os
# Run export OPENAI_API_KEY=sk-YOUR_OPENAI_API_KEY...
# Get openAI api key by reading local .env file
from dotenv import load_dotenv, find_dotenv
dotenv_path = os.path.abspath('../../../.env')
_ = load_dotenv(dotenv_path)
OPENAI_API_KEY  = os.environ['OPENAI_API_KEY']
PG_DB_PW = os.environ['POSTGRES_DB_PASSWORD']
host= os.environ['POSTGRES_DB_HOST']
port= os.environ['POSTGRES_DB_PORT']
user= os.environ['POSTGRES_DB_USER']
password= os.environ['POSTGRES_DB_PASSWORD']
dbname= os.environ['POSTGRES_DB_DBNAME']


In [2]:
import openai
import os
import pandas as pd
import numpy as np
import json
import tiktoken
import psycopg2
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

from IPython.display import Markdown, display

import pandas as pd

from langchain.vectorstores.pgvector import PGVector
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import ChatOpenAI, OpenAI
from langchain.chains import RetrievalQA



/var/folders/s4/l2klswfj1wvcw_vbmvvbc70h0000gn/T/ipykernel_60761/3406201478.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# CONNECTION_STRING = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
# CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}?sslmode=require"
CONNECTION_STRING = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
collection_name= "arxiv"


In [ ]:
conn = psycopg2.connect(
    host=host,
    port=port,
    user=user,
    password=PG_DB_PW,
    database=dbname
)

In [ ]:
# Connect to PostgreSQL database in Timescale using connection string
conn = psycopg2.connect(CONNECTION_STRING)
cur = conn.cursor()

#install pgvector
cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
conn.commit()

In [ ]:
embedding_df = pd.read_csv('../../data-ingest/data/embeddings/test_embedding.csv', index_col=0)

In [ ]:
embedding_df

In [ ]:
#load documents from Pandas dataframe for insertion into database
from langchain.document_loaders import DataFrameLoader

# page_content_column is the column name in the dataframe to create embeddings for
loader = DataFrameLoader(embedding_df, page_content_column = 'content')
docs = loader.load()


In [ ]:
type(docs[0])

In [1]:
embeddings = OpenAIEmbeddings()

NameError: name 'OpenAIEmbeddings' is not defined

In [ ]:
embedding_df['content']

In [ ]:
from langchain.vectorstores.pgvector import DistanceStrategy

db = PGVector.from_documents(
    documents=docs,
    embedding=embeddings,
    collection_name=collection_name,
    connection_string=CONNECTION_STRING,
    distance_strategy=DistanceStrategy.COSINE,
    pre_delete_collection=False 
)

In [ ]:
PGVector.__dict__

In [ ]:
from langchain.schema import Document

# Query for which we want to find semantically similar documents
query = "What is a quasar?"

#Fetch the k=3 most similar documents
docs =  db.similarity_search(query, k=3)


In [ ]:
docs

In [ ]:
# Create retriever from database
# We specify the number of results we want to retrieve (k=3)
retriever = db.as_retriever(
    search_kwargs={"k": 3}
    )


In [ ]:
llm = ChatOpenAI(temperature = 0.0, model = 'gpt-3.5-turbo-16k')


In [ ]:
from langchain.chains import RetrievalQA
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    verbose=True,
)


In [ ]:
query =  "What is a quasar?"

response = qa_stuff.run(query)

from IPython.display import Markdown, display
display(Markdown(response))


In [ ]:
# New chain to return context and sources
qa_stuff_with_sources = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

query =  "What is the evidence for an enhanced accretion episode from Sgr A* in 2019?"

# To run the query, we use a different syntax since we're returning more than just the response text
responses = qa_stuff_with_sources({"query": query})


In [ ]:
source_documents = responses["source_documents"]
source_content = [doc.page_content for doc in source_documents]
source_metadata = [doc.metadata for doc in source_documents]

# Construct a single string with the LLM output and the source titles and urls
def construct_result_with_sources():
    result = responses['result']
    result += "\n\n"
    result += "Sources used:"
    for i in range(len(source_content)):    
        result += "\n\n"
        result += source_metadata[i]['title']
        result += "\n\n"
        return result

display(Markdown(construct_result_with_sources()))


In [ ]:
query =  "What distinguishes a quasar from a normal black hole?"

# To run the query, we use a different syntax since we're returning more than just the response text
responses = qa_stuff_with_sources({"query": query})

source_documents = responses["source_documents"]
source_content = [doc.page_content for doc in source_documents]
source_metadata = [doc.metadata for doc in source_documents]

# Construct a single string with the LLM output and the source titles and urls
def construct_result_with_sources():
    result = responses['result']
    result += "\n\n"
    result += "Sources used:"
    for i in range(len(source_content)):    
        result += "\n\n"
        result += source_metadata[i]['title']
        result += "\n\n"
        return result

display(Markdown(construct_result_with_sources()))


In [ ]:
# db = PGVector.from_documents(
#     embedding=embeddings,
#     documents=docs,
#     collection_name=collection_name,
#     connection_string=CONNECTION_STRING,
#     pre_delete_collection=False,
# )


In [28]:
store = PGVector(
    collection_name=collection_name,
    connection_string=CONNECTION_STRING,
    embedding_function=embeddings,
)

In [14]:
pdf_texts_df = pd.read_csv('../../data-ingest/data/text/pdf_texts.csv', index_col=0)

In [16]:
pdf_texts_df['content']

0    Detection of stellar light from quasar host ga...
1    Quasars and the\nIntergalactic Medium at\nCosm...
2    Draft version August 22, 2023\nTypeset using L...
Name: content, dtype: object

In [19]:
store.add_texts(pdf_texts_df['content'])

['06f9f18a-bfa6-11ee-8a3d-9801a78f9833',
 '06f9fdf6-bfa6-11ee-8a3d-9801a78f9833',
 '06f9fee6-bfa6-11ee-8a3d-9801a78f9833']

In [20]:
schema = {
    "properties": {
        "article_title": {"type": "string"},
        "authors": {"type": "string"},
        "publish_date": {"type": "string"},
    },
    "required": ["article_title", "authors", "publish_date"],
}

In [29]:
store.add_texts(pdf_texts_df['content'], metadatas=schema)

['113d7142-bfa8-11ee-8a3d-9801a78f9833',
 '113d72c8-bfa8-11ee-8a3d-9801a78f9833',
 '113d7340-bfa8-11ee-8a3d-9801a78f9833']

In [26]:
store.delete_collection()

In [ ]:
store

In [ ]:
store.add_documents(docs)

In [ ]:
retriever = store.as_retriever()
llm = ChatOpenAI(temperature = 0.0, model = 'gpt-3.5-turbo-16k')


In [ ]:
# New chain to return context and sources
qa_stuff_with_sources = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

query =  "What is the evidence for an enhanced accretion episode from Sgr A* in 2019?"

# To run the query, we use a different syntax since we're returning more than just the response text
responses = qa_stuff_with_sources({"query": query})


In [ ]:
source_documents = responses["source_documents"]
source_content = [doc.page_content for doc in source_documents]
source_metadata = [doc.metadata for doc in source_documents]

# Construct a single string with the LLM output and the source titles and urls
def construct_result_with_sources():
    result = responses['result']
    result += "\n\n"
    result += "Sources used:"
    for i in range(len(source_content)):    
        result += "\n\n"
        result += source_metadata[i]['title']
        result += "\n\n"
        return result

display(Markdown(construct_result_with_sources()))